In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

with open("carteira.txt", "r") as file:
    text = file.readlines()

wallet = {}

for line in text:
    ticker, value = line.split("-")
    ticker = f"{ticker.strip()}.SA"
    value = float(value.strip())
    wallet[ticker] = value

ticker_names = list(wallet.keys())

ticker_names.append("^BVSP")

initial_date = "2024-11-01"
final_date = "2025-12-31"
tickers = yf.Tickers(ticker_names)

tbl = tickers.download(start=initial_date, end=final_date)['Close']

try:
    from IPython.display import display
    display(tbl)  
except ImportError:
    print(tbl)  

rents = {}
for ticker in tbl.columns:
    rents[ticker] = tbl[ticker].iloc[-1] / tbl[ticker].iloc[0] - 1

initial_value = sum(wallet.values())
final_value = sum(wallet[ticker] * (1 + rents[ticker]) for ticker in wallet)
wallet_rent = final_value / initial_value - 1

df_resultados = pd.DataFrame({
    "Valor Investido (R$)": wallet,
    "Preço Inicial (R$)": {t: tbl[t].iloc[0] for t in wallet},
    "Preço Final (R$)": {t: tbl[t].iloc[-1] for t in wallet},
    "Rentabilidade (%)": {t: rents[t] * 100 for t in wallet}
}).round(2)

print("\nDetalhes da Carteira de Investimentos:\n")
try:
    display(df_resultados) 
except NameError:
    print(df_resultados)  

print(f"\nRentabilidade total da carteira: {wallet_rent:.2%}")

if "^BVSP" in rents:
    print(f"Rentabilidade do IBOV: {rents['^BVSP']:.2%}")
    if wallet_rent > rents["^BVSP"]:
        print("A carteira *GANHOU* do IBOV!")
    else:
        print("A carteira *PERDEU* para o IBOV.")
else:
    print("Não foi possível obter a rentabilidade do IBOV.")

tbl.plot(title="Evolução dos Ativos da Carteira")
plt.xlabel("Data")
plt.ylabel("Preço Ajustado (R$)")
plt.legend(title="Ativos", loc="upper left", bbox_to_anchor=(1, 1))
plt.grid()
plt.tight_layout()

plt.show()